In [120]:
# Librería para comando de sistema
import os
# Librerías para manejo de datos
import pandas as pd
# Para realizar la separación del conjunto de aprendizaje en entrenamiento y test.
from sklearn.model_selection import train_test_split
# Para la creación de modelo Ridge
from sklearn.linear_model import Ridge
# Para búsqueda de hiperparámetros
from sklearn.model_selection import GridSearchCV
# Para la validación cruzada
from sklearn.model_selection import KFold 
# Para determinar el rendimiento del modelo con las métricas MSE, MAE y R2
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline

In [121]:
data = pd.read_excel("datos_proyecto_modulo2.xlsx")

In [122]:
# Mostrar los datos
data.head()

,area_quemada,clase_incendio,mes_incendio,vegetacion,Prec_pre_30,Prec_pre_15,Prec_pre_7,Prec_cont
0,3.0,B,Diciembre,Desierto polar_roca,59.8,8.4,0.0,86.8
1,60.0,C,Febrero,Bosque tropical perennifolio secundario,168.8,42.2,18.1,124.5
2,1.0,B,Junio,Bosque tropical perennifolio,10.4,7.2,0.0,0.0
3,5.2,B,Enero,Matorral abierto,26.0,0.0,0.0,0.0
4,1.0,B,Noviembre,Matorral abierto,28.4,27.5,1.2,55.4


In [123]:
# Es recomendable que todos los pasos de preparación se realicen sobre otro archivo.
data_t = data
# Eliminación data vacía.
data_t=data_t.dropna()
# Eliminación de registros duplicados.
data_t=data_t.drop_duplicates()
# Transformación de los datos categóricos a numéricos.

data_t = pd.get_dummies(data_t, columns=['clase_incendio', 'mes_incendio', 'vegetacion'], dtype=int)

In [124]:
# Se selecciona la variable objetivo, en este caso "precio".
Y=data_t['area_quemada']
# Del conjunto de datos se elimina la variable "precio".
X=data_t.drop(['area_quemada'], axis=1)

In [125]:
# Se realiza la división entrenamiento - test. Se deja 20% de los datos para el test.
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20, random_state = 0)

In [126]:
#Como ya has podido conocer la regresión Ridge utiliza la norma L2 del vector de coeficientes como término de regularización. 
#Para controlar la cantidad de regularización se utiliza el hiperparámetro alpha. Probemos con un valor igual a 0.1.
modelo_ridge = make_pipeline(StandardScaler(), Ridge(alpha=0.1))
modelo_ridge      # Se muestra el modelo

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('ridge', Ridge(alpha=0.1))])

In [127]:
# Ajuste del modelo
modelo_ridge.fit(X_train,Y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('ridge', Ridge(alpha=0.1))])

In [128]:
# Ahora probemos el rendimiento sobre el conjunto test.
y_pred = modelo_ridge.predict(X_test)
print("RMSE: %.2f" % mean_squared_error(Y_test, y_pred, squared=False))
print("MAE: %.2f" % mean_absolute_error(Y_test, y_pred))
print('R²: %.2f' % r2_score(Y_test, y_pred))

RMSE: 17241.30
MAE: 3646.29
R²: 0.23


c:\Users\harve\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [129]:
# Dividimos el conjunto de entrenamiento en dos: una para la construcción del modelo (sería el nuevo conjunto de 
# entrenamiento) y otro para la validación, el cuál será utilizado para determinar el rendimiento del modelo con una
# combinación específica de hiperparámetros.
X_trainval, X_val, Y_trainval, Y_val = train_test_split(X_train, Y_train, test_size = 0.15, random_state = 0)

In [130]:
## Fijemos un valor de alpha en 1, construyamos el modelo y probemos sobre validación. Utilicemos MAE como métrica de
# rendimiento para hacer la selección.
modelo_ridge = make_pipeline(StandardScaler(), Ridge(alpha=0.1))
modelo_ridge.fit(X_trainval,Y_trainval)
y_pred = modelo_ridge.predict(X_val)
print("MAE: %.2f" % mean_absolute_error(Y_val, y_pred))

MAE: 3167.76


In [131]:
## Fijemos un valor de alpha en 2, construyamos el modelo y probemos sobre validación. Utilicemos MAE como métrica de
# rendimiento para hacer la selección.
modelo_ridge = make_pipeline(StandardScaler(), Ridge(alpha=0.5))
modelo_ridge.fit(X_trainval,Y_trainval)
y_pred = modelo_ridge.predict(X_val)
print("MAE: %.2f" % mean_absolute_error(Y_val, y_pred))

MAE: 3167.72


In [132]:
## Fijemos un valor de alpha en 5, construyamos el modelo y probemos sobre validación. Utilicemos MAE como métrica de
# rendimiento para hacer la selección.
modelo_ridge = make_pipeline(StandardScaler(), Ridge(alpha=1))
modelo_ridge.fit(X_trainval,Y_trainval)
y_pred = modelo_ridge.predict(X_val)
print("MAE: %.2f" % mean_absolute_error(Y_val, y_pred))

MAE: 3167.68


El mejor parametro con valor MAE ES alpha = 1

In [133]:
modelo_ridge = make_pipeline(StandardScaler(), Ridge(alpha=1))
modelo_ridge.fit(X_train,Y_train)
y_pred = modelo_ridge.predict(X_test)
print("RMSE: %.2f" % mean_squared_error(Y_test, y_pred, squared=False))
print("MAE: %.2f" % mean_absolute_error(Y_test, y_pred))
print('R²: %.2f' % r2_score(Y_test, y_pred))

RMSE: 17241.32
MAE: 3646.24
R²: 0.23


c:\Users\harve\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Ahora con validacion cruzada con K particiones

In [134]:
# Fijemos el número de particiones. Utilizaremos K = 10.
particiones = KFold(n_splits=10, shuffle=True, random_state = 0)

In [135]:
# Ahora tenemos que definir el espacio de búsqueda, es decir, los valores de alpha que queremos que sean considerados. 
# Para esto se define un diccionario (o grilla) con los valores que podrá asumir el hiperparámetro alpha.
# Probemos con los siguientes valores:
param_grid = {'ridge__alpha': [0.1,0.5,1]}

In [136]:
# Definimos el modelo sin ningún valor del hiperparámetro alpha
modelo_ridge1 = Pipeline([
    ('scaler', StandardScaler()),  # Normalización
    ('ridge', Ridge())  # Modelo Ridge
])

In [137]:
# Ahora utilizamos GridSearch sobre el grid definido y con 10 particiones en la validación cruzada.
mejor_modelo = GridSearchCV(modelo_ridge1, param_grid, cv=particiones, n_jobs=-1)
# Ajuste del modelo
mejor_modelo.fit(X_train, Y_train)

GridSearchCV(cv=KFold(n_splits=10, random_state=0, shuffle=True),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('ridge', Ridge())]),
             n_jobs=-1, param_grid={'ridge__alpha': [0.1, 0.5, 1]})

In [138]:
# Podemos ver cual fue el resultado de la búsqueda (mejor valor de alpha)
print("Mejor parámetro: {}".format(mejor_modelo.best_params_))

Mejor parámetro: {'ridge__alpha': 1}


In [139]:
# También puedes indicarle a GridSearch que seleccione el mejor modelo a partir de la búsqueda con base en una métrica 
# particular. Por ejemplo, hubiésemos podido utilizar la siguiente línea de comando:
mejor_modelo = GridSearchCV(modelo_ridge, param_grid, scoring = 'neg_root_mean_squared_error', n_jobs=-1)
mejor_modelo.fit(X_train, Y_train)
print("Mejor parámetro: {}".format(mejor_modelo.best_params_)) 

Mejor parámetro: {'ridge__alpha': 1}


In [140]:
modelo_final = mejor_modelo.best_estimator_
# Probemos ahora este modelo sobre test.
y_pred = modelo_final.predict(X_test)
print("RMSE: %.2f" % mean_squared_error(Y_test, y_pred, squared=False))
print("MAE: %.2f" % mean_absolute_error(Y_test, y_pred))
print('R²: %.2f' % r2_score(Y_test, y_pred))

RMSE: 17241.32
MAE: 3646.24
R²: 0.23


c:\Users\harve\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
